In [8]:
import pandas as pd
import yfinance as yf

In [9]:
sp500_tickers = ['MCD', 'SBUX', 'KO', 'QSR', 'PEP'] #sample just for testing purposes
all_companies_data = {}
historical_prices_data = {}
financial_statements_data = {}

for ticker in sp500_tickers:
    print(f"Fetching data for {ticker}...")
    try: #using try in case data not available for one of the tickers - so program wont crash, it will go to the except line
        stock = yf.Ticker(ticker)

        # 1. Fetch Company Info (for Stock Screener)
        info = stock.info
        all_companies_data[ticker] = {
            'Name': info.get('longName', 'N/A'), #returns 'N/A' if 'longname'/any other info doesn't exist
            'Sector': info.get('sector', 'N/A'),
            'Industry': info.get('industry', 'N/A'),
            'MarketCap': info.get('marketCap', 'N/A'),
            'PE_Ratio': info.get('trailingPE', 'N/A'),
            'DividendYield': info.get('dividendYield', 'N/A'),
            'RevenueGrowth': info.get('revenueGrowth', 'N/A'), # TTM revenue growth?
            'ProfitMargin': info.get('profitMargins', 'N/A'), # TTM profit margins?
            'DebtToEquity': info.get('debtToEquity', 'N/A'),
            'CurrentRatio': info.get('currentRatio', 'N/A'),
            'ReturnOnEquity': info.get('returnOnEquity', 'N/A'),
        }

        # 2. Stock Price data
        hist = stock.history(period="1y") #can change to a different period?
        historical_prices_data[ticker] = hist

        
        financial_statements_data[ticker] = {
            'Income Statement': stock.financials.T, #stock.financials retrieves the annual income statements for the ticker as a Pandas DataFrame. 
            'Balance Sheet': stock.balance_sheet.T, #stock.balance_sheet retrieves the annual balamce sheets
            'Cash Flow': stock.cashflow.T #stock.cashflow retrieves annual cashflow statements
        } #.T to transpose so that  dates are in rows and financial data in columns (to help make data easier to work with), and then we store these 3 financial statemen's info in financial_statements_data[ticker]


        print(f"Successfully fetched data for {ticker}")

    except Exception as e: #assigns any errors to variable e
        print(f"Could not fetch data for {ticker}: {e}") #prints error for us

sp500_df_for_screener = pd.DataFrame.from_dict(all_companies_data, orient='index') #converts dictionary to pandas data frame (from #1 company info)

print("\nQuick summary of data")
print("\nS&P 500 Screener Data (first 5 rows):")
print(sp500_df_for_screener.head())

print("\nHistorical Prices for First stock (MCD) (first 5 rows):")
if 'MCD' in historical_prices_data:
    print(historical_prices_data['MCD'].head())

print("\nAnnual Income Statement for First stock (MCD) (first 5 rows/columns):")
if 'MCD' in financial_statements_data and 'Income Statement' in financial_statements_data['MCD']:
    print(financial_statements_data['MCD']['Income Statement'].head())

Fetching data for MCD...
Successfully fetched data for MCD
Fetching data for SBUX...
Successfully fetched data for SBUX
Fetching data for KO...
Successfully fetched data for KO
Fetching data for QSR...
Successfully fetched data for QSR
Fetching data for PEP...
Successfully fetched data for PEP

Quick summary of data

S&P 500 Screener Data (first 5 rows):
                                      Name              Sector  \
MCD                 McDonald's Corporation   Consumer Cyclical   
SBUX                 Starbucks Corporation   Consumer Cyclical   
KO                   The Coca-Cola Company  Consumer Defensive   
QSR   Restaurant Brands International Inc.   Consumer Cyclical   
PEP                          PepsiCo, Inc.  Consumer Defensive   

                       Industry     MarketCap   PE_Ratio  DividendYield  \
MCD                 Restaurants  209518968832  25.839504           2.42   
SBUX                Restaurants  108242100224  34.636364           2.66   
KO    Beverages - Non

In [10]:
import jupyter_ai
import pandas as pd
import numpy as np
import datetime
import yfinance as yfin
import matplotlib.pyplot as plt
%reload_ext jupyter_ai
%ai list ollama
%config AiMagics.default_language_model = "ollama:gemma3"

In [11]:
%%ai
"whats 2+2"

4


In [12]:
example_stock_info = sp500_df_for_screener.loc['MCD'].to_string()

In [13]:
def calculate_fundamental_indicators(financials):
    import pandas as pd
    import numpy as np

    # Helper: Try to standardize the date column across statements
    def standardize_date_column(df, label):
        date_col = next((col for col in df.columns if 'date' in col.lower()), None)
        if date_col is None:
            raise KeyError(f"No date-like column found in {label} for merging.")
        df = df.rename(columns={date_col: 'Date'})
        return df

    # Extract individual statements
    income_df = financials.get('income_statement', pd.DataFrame())
    balance_df = financials.get('balance_sheet', pd.DataFrame())
    cashflow_df = financials.get('cash_flow', pd.DataFrame())

    # Standardize and copy
    income = standardize_date_column(income_df.copy(), 'Income Statement')
    balance = standardize_date_column(balance_df.copy(), 'Balance Sheet')
    cashflow = standardize_date_column(cashflow_df.copy(), 'Cash Flow Statement')

    # Perform outer merges to preserve all data
    df = income.merge(balance, on='Date', how='outer').merge(cashflow, on='Date', how='outer')

    # Sort by date for clean chronological output
    df.sort_values('Date', inplace=True)

    # Optional: Fill zeros or NaNs based on context
    df.replace({0: np.nan}, inplace=True)  # avoid divide-by-zero in ratio calculations

    # Normalize column names to consistent snake_case
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(r"[^\w_]", "", regex=True)

    # Debug: check actual columns
    # print("Merged columns:", df.columns.tolist())

    # Map actual column names
    df['Net_Profit_Margin'] = df['Net_Income_Common_Stockholders'] / df['Total_Revenue']
    df['Operating_Margin'] = df['Operating_Income'] / df['Total_Revenue']
    df['Gross_Margin'] = df['Gross_Profit'] / df['Total_Revenue']

    df['ROA'] = df['Net_Income_Common_Stockholders'] / df['Total_Assets']
    df['ROE'] = df['Net_Income_Common_Stockholders'] / df['Total_Equity']

    df['Current_Ratio'] = df['Current_Assets'] / df['Current_Liabilities']
    df['Quick_Ratio'] = (df['Current_Assets'] - df['Inventory']) / df['Current_Liabilities']


In [14]:
import pandas as pd
import numpy as np

def calculate_fundamental_indicators(financials):
    # Extract individual statements
    income_df = financials.get('income_statement', pd.DataFrame()).copy()
    balance_df = financials.get('balance_sheet', pd.DataFrame()).copy()
    cashflow_df = financials.get('cash_flow', pd.DataFrame()).copy()

    # Move index to column for merging
    for df in [income_df, balance_df, cashflow_df]:
        df['Date'] = df.index
        df.reset_index(drop=True, inplace=True)

    # Merge on 'Date' across statements
    df = income_df.merge(balance_df, on='Date', how='outer').merge(cashflow_df, on='Date', how='outer')

    # Avoid divide-by-zero errors
    df.replace({0: np.nan}, inplace=True)

    # Normalize column names (snake_case for consistent access)
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace(r"[^\w_]", "", regex=True)

    print("Columns after merge and normalization:")
    print(df.columns.tolist())

    print("\nSample data (first 3 rows):")
    print(df[['Date'] + [col for col in df.columns if 'Income' in col or 'Revenue' in col or 'Assets' in col or 'Equity' in col]].head(3))


    # === Profitability Ratios ===
    df['Net_Profit_Margin'] = df['Net_Income_Common_Stockholders'] / df['Total_Revenue']
    df['Operating_Margin'] = df['Operating_Income'] / df['Total_Revenue']
    df['Gross_Margin'] = df['Gross_Profit'] / df['Total_Revenue']

    # === Return Ratios ===
    df['ROA'] = df['Net_Income_Common_Stockholders'] / df['Total_Assets']
    df['ROE'] = df['Net_Income_Common_Stockholders'] / df['Stockholders_Equity']

    # === Liquidity Ratios ===
    df['Current_Ratio'] = df['Current_Assets'] / df['Current_Liabilities']
    df['Quick_Ratio'] = (df['Current_Assets'] - df['Inventory']) / df['Current_Liabilities']

    # === Leverage Ratios ===
    df['Debt_to_Equity'] = df['Total_Debt'] / df['Stockholders_Equity']
    df['Debt_to_Assets'] = df['Total_Debt'] / df['Total_Assets']

    # === Cash Flow Ratios ===
    df['Operating_Cash_Flow_Ratio'] = df['Operating_Cash_Flow'] / df['Current_Liabilities']
    df['Free_Cash_Flow'] = df['Operating_Cash_Flow'] - df['Capital_Expenditure']

    # df.dropna(inplace=True)

    return df


In [ ]:
def generate_fundamental_dcf_prompt(ticker: str, df_with_fundamentals: pd.DataFrame) -> str:
    recent_data = df_with_fundamentals.tail(4)  # Use last 4 periods (e.g., quarters or years)

    # Format relevant rows to string for visibility in the prompt
    data_cols = [
        'Total_Revenue', 'Net_Income_Common_Stockholders', 'Operating_Income', 'Gross_Profit',
        'Operating_Cash_Flow', 'Capital_Expenditure', 'Total_Assets',
        'Stockholders_Equity', 'Total_Debt', 'Current_Assets', 'Current_Liabilities'
    ]
    fundamental_str = recent_data[data_cols].to_string(index=False)

    # Prompt construction
    prompt = f"""
Perform a Discounted Cash Flow (DCF) analysis for {ticker} using the following recent fundamental financial data:

{fundamental_str}

Please:

1. **Project Free Cash Flows (FCF)** for the next 5 years based on historical Operating Cash Flow minus Capital Expenditures.
2. **Calculate the Terminal Value** using the Gordon Growth Model. Use an assumed long-term growth rate (e.g., 2.5%) and a discount rate (e.g., 8%). Show the formula and exact math.
3. **Discount all cash flows (5-year FCF + Terminal Value)** to present value.
4. **Compute Enterprise Value (EV)** and then adjust it to get Equity Value using:
   [
   text{{Equity Value}} = text{{Enterprise Value}} - text{{Net Debt}}
   ]
   where Net Debt = Total Debt - Cash (if available, assume 0 if not).
5. **Determine the intrinsic stock price** using number of shares outstanding (assume a number or request if unknown).
6. **Summarize the DCF in a table**, showing:
   - Year
   - Projected FCF
   - Present Value of FCF
   - Cumulative PV
   - Terminal Value
   - Enterprise Value
   - Equity Value
   - Implied Share Price

Then:

- Provide a concise fundamental analysis of the company's **profitability, efficiency, liquidity, and leverage** using ratios like:
  - Net Profit Margin
  - ROE and ROA
  - Current Ratio and Quick Ratio
  - Debt-to-Equity

Finally:

- Offer a valuation opinion: Is the stock **undervalued**, **overvalued**, or **fairly valued** relative to its calculated intrinsic value?

Use all available numeric data to justify your answer.
"""

    return prompt


In [16]:
if 'MCD' in financial_statements_data:
    # Extract the raw financials for MCD
    mcd_financials_raw = financial_statements_data['MCD']

    # Copy and fix the structure so the index becomes a 'Date' column
    income_df = mcd_financials_raw['Income Statement'].copy()
    balance_df = mcd_financials_raw['Balance Sheet'].copy()
    cashflow_df = mcd_financials_raw['Cash Flow'].copy()

    for df in [income_df, balance_df, cashflow_df]:
        df['Date'] = df.index
        df.reset_index(drop=True, inplace=True)

    # Reassemble the corrected input structure using lowercase keys
    mcd_financials_fixed = {
        'income_statement': income_df,
        'balance_sheet': balance_df,
        'cash_flow': cashflow_df
    }

    # Now safely pass into the calculation function
    mcd_fundamental_data = calculate_fundamental_indicators(mcd_financials_fixed)

    if not mcd_fundamental_data.empty:
        ai_dcf_prompt = generate_fundamental_dcf_prompt('MCD', mcd_fundamental_data)

        print("\nAI prompt for Fundamental DCF Analysis ---")
        print(ai_dcf_prompt)
    else:
        print("Cannot generate DCF analysis prompt: MCD fundamental data is empty after processing.")
else:
    print("Cannot generate DCF analysis prompt: MCD not found in financial statements data.")


Columns after merge and normalization:
['Tax_Effect_Of_Unusual_Items', 'Tax_Rate_For_Calcs', 'Normalized_EBITDA', 'Total_Unusual_Items', 'Total_Unusual_Items_Excluding_Goodwill', 'Net_Income_From_Continuing_Operation_Net_Minority_Interest', 'Reconciled_Depreciation', 'Reconciled_Cost_Of_Revenue', 'EBITDA', 'EBIT', 'Net_Interest_Income', 'Interest_Expense', 'Interest_Income', 'Normalized_Income', 'Net_Income_From_Continuing_And_Discontinued_Operation', 'Total_Expenses', 'Rent_Expense_Supplemental', 'Total_Operating_Income_As_Reported', 'Diluted_Average_Shares', 'Basic_Average_Shares', 'Diluted_EPS', 'Basic_EPS', 'Diluted_NI_Availto_Com_Stockholders', 'Net_Income_Common_Stockholders', 'Net_Income', 'Net_Income_Including_Noncontrolling_Interests', 'Net_Income_Continuous_Operations', 'Tax_Provision', 'Pretax_Income', 'Other_Income_Expense', 'Other_Non_Operating_Income_Expenses', 'Special_Income_Charges', 'Gain_On_Sale_Of_Ppe', 'Gain_On_Sale_Of_Business', 'Write_Off', 'Impairment_Of_Capital

/var/folders/gj/th4g1lws1_99nfx8psc2vckm0000gn/T/ipykernel_90436/534330806.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({0: np.nan}, inplace=True)


In [18]:
%%ai
{ai_dcf_prompt}

Okay, let's perform a DCF analysis for McDonald's (MCD) based on the provided data.  Due to the missing data points, we'll make some reasonable assumptions and highlight the limitations of this analysis.

**1. Projected Free Cash Flows (FCF) - 5 Years**

We'll use Operating Cash Flow (OCF) minus Capital Expenditures (CapEx) as the basis for FCF projections.  We'll assume constant growth for simplicity.  Let's use the average OCF figure (2.322290e+10) as the base.

| Year | Projected OCF (USD Billion) | CapEx (USD Billion) | Projected FCF (USD Billion) |
|------|-----------------------------|-----------------------|-----------------------------|
| 2024 | 2.322290                    | -2.040000           | 0.282290                   |
| 2025 | 2.322290                    | -2.040000           | 0.282290                   |
| 2026 | 2.322290                    | -2.040000           | 0.282290                   |
| 2027 | 2.322290                    | -2.040000           | 0.282290                   |
| 2028 | 2.322290                    | -2.040000           | 0.282290                   |


**2. Terminal Value Calculation**

*   **Formula:** Terminal Value = (FCF<sub>2028</sub> * (1 + g)) / (r - g)
    Where:
    *   FCF<sub>2028</sub> = 0.282290 Billion
    *   g = Long-term growth rate (2.5%) = 0.025
    *   r = Discount Rate (8%) = 0.08

*   **Calculation:**  Terminal Value = (0.282290 * (1 + 0.025)) / (0.08 - 0.025) = 0.282290 * 1.025 / 0.055 = 0.562922 Billion

**3. Discounting Cash Flows**

We'll discount all cash flows using the 8% discount rate.

| Year | Projected FCF (USD Billion) | Discount Factor (1 / (1 + r)) | Present Value of FCF (USD Billion) | Cumulative PV (USD Billion) |
|------|-----------------------------|-----------------------------|-----------------------------------|-----------------------------|
| 2024 | 0.282290                    | 0.9259256              | 0.257447                             | 0.257447                     |
| 2025 | 0.282290                    | 0.8594783              | 0.240849                            | 0.500000                     |
| 2026 | 0.282290                    | 0.7980482              | 0.223463                           | 0.746863                     |
| 2027 | 0.282290                    | 0.7403859              | 0.207629                          | 0.955492                     |
| 2028 | 0.282290                    | 0.6830135              | 0.193345                           | 2.148837                     |
| Terminal Value | 0.562922                    | 0.6211602              | 0.351363                          | 2.499999                     |
| **Total PV** |                          |                            | **6.911740**                 |                              |



**4. Enterprise Value & Equity Value**

*   **Enterprise Value (EV)** = Total PV (6.911740 Billion)
*   **Net Debt** = Total Debt - Cash. We'll assume Cash = 0.
*   **Equity Value** = EV - Net Debt = 6.911740 - 0 = 6.911740 Billion

**5. Implied Share Price**

*   **Number of Shares Outstanding** (Assumption): Let's assume 798,600,000. This is the current number of shares outstanding.
*   **Implied Share Price** = Equity Value / Number of Shares = 6.911740 / 798,600,000 = 0.00000857  = ~$0.000857 or $0.00086

**6. Summary Table**

| Year | Projected FCF (USD Billion) | Present Value of FCF (USD Billion) | Cumulative PV (USD Billion) | Terminal Value (USD Billion) | Enterprise Value (USD Billion) | Equity Value (USD Billion) | Implied Share Price (USD) |
|------|-----------------------------|-----------------------------------|-----------------------------|------------------------------|------------------------------|-----------------------------|---------------------------|
| 2024 | 0.282290                    | 0.257447                          | 0.257447                     |                              | 6.911740                    | 6.911740                    | 0.00086                     |
| 2025 | 0.282290                    | 0.240849                          | 0.500000                    |                              | 6.911740                    | 6.911740                    | 0.00086                     |
| 2026 | 0.282290                    | 0.223463                          | 0.746863                     |                              | 6.911740                    | 6.911740                    | 0.00086                     |
| 2027 | 0.282290                    | 0.207629                          | 0.955492                     |                              | 6.911740                    | 6.911740                    | 0.00086                     |
| 2028 | 0.282290                    | 0.193345                          | 2.148837                     | 0.562922                     | 6.911740                    | 6.911740                    | 0.00086                     |
| Terminal| 0.562922                    | 0.351363                          | 2.499999                    |                              | 6.911740                    | 6.911740                    | 0.00086                     |


**7. Fundamental Analysis**

*   **Profitability:**
    *   **Net Profit Margin:**  Based on the provided numbers, the Net Profit Margin is approximately 61.77%.  This is considered healthy.
    *   **ROE & ROA:**  Calculating these accurately requires more data.  However, given the high Net Profit Margin, both ROE and ROA would likely be high.
*   **Efficiency:** MCD is very efficient based on its profitability.
*   **Liquidity:**  This analysis doesn't provide liquidity ratios as sufficient data is missing.
*   **Leverage:** MCD’s debt levels would need to be analyzed to determine its leverage ratio.

**8. Valuation Opinion**

Based on this highly simplified DCF analysis, the implied share price is only $0.00086.  This indicates the stock is **significantly undervalued** relative to the calculated intrinsic value.

**Important Disclaimer:** This analysis is highly sensitive to the assumptions made, especially the discount rate, growth rate, and number of shares outstanding. The missing data makes this a very rough estimate. A more thorough analysis would require significantly more detailed data and a more sophisticated model.  This is for informational purposes only and should not be considered investment advice.